# M1M3 Force Actuator following errors FFT

In [ ]:
# Times Square parameters
t_start = "2025-05-27T18:30:15"
t_end = "2025-05-27T18:30:24"

> **Note:** Please select a short time range when running this code.  
> For example:
> ```python
> t_start = "2025-05-27T18:30:15"
> t_end = "2025-05-27T18:30:24"
> ```
> In this example, the time range is only 9 seconds, but the code may still take a while to run.  
> Selecting longer time intervals (e.g., several minutes or hours) can significantly increase execution time.
>
> In the future, the code will be optimized to efficiently handle full observation days.
>
> Additionally, future versions will reduce the number of plots shown by filtering out force actuators with no
> significant behavior, helping focus the analysis on potentially problematic components.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import argparse
from astropy.time import Time, TimeDelta

In [ ]:
from lsst.summit.utils.tmaUtils import TMAEventMaker
from lsst.summit.utils.efdUtils import EfdClient, getEfdData, makeEfdClient, getDayObsEndTime, getDayObsStartTime
from lsst.ts.xml.tables.m1m3 import FATable

In [ ]:
# Create an EFD client instance
client = makeEfdClient()

# Define the start and end time
t_start = Time(t_start)
t_end = Time(t_end)

main(t_start, t_end)


In [ ]:
def loop_over_actuators(client, topic, nb_actuators, t_start, t_end):
    print(f"Processing {topic}")

    plot_directory = "./plots/"
    if not os.path.exists(plot_directory):
        os.makedirs(plot_directory)
    FA_error = [f"{topic}{i}" for i in range(nb_actuators)]

    if "secondary" in topic:
        secondary_actuator_id = np.array([])
        for i in range(len(FATable)):
            if FATable[i].s_index is not None:
                secondary_actuator_id = np.append(
                    secondary_actuator_id, FATable[i].actuator_id
                )
        secondary_actuator_id = secondary_actuator_id.astype(int)
        
    for fa in range(nb_actuators):
        if fa % 10 == 0:
            print(f"{fa}/{nb_actuators}")
        df = getEfdData(
            client,
            "lsst.sal.MTM1M3.forceActuatorData",
            columns=FA_error,
            begin=t_start,
            end=t_end,
        )
        
        dt = (df[f"{topic}0"].index[1] - df[f"{topic}0"].index[0]).total_seconds()
        freqs = np.fft.fftfreq(len(df), d=dt)
        positive_mask = freqs > 0
        fft_frequency = freqs[positive_mask]
        fft_result = np.fft.fft(df.values, axis=0)
        fft_magnitudes = np.abs(fft_result[positive_mask, :])
        plt.figure(figsize=(10, 5))
        
        if "primary" in topic:
            label = FATable[fa].actuator_id
        else:
            label = secondary_actuator_id[fa]
            
        plt.plot(
            fft_frequency, fft_magnitudes[:, fa], label=f"Primary actuator {label}"
        )
        plt.title(f"Power spectrum from {t_start} to {t_end}")
        plt.xlabel("Frequency [Hz]")
        plt.ylabel("Magnitude")
        plt.legend()
        plt.show()

        #if "primary" in topic:
        #    plt.savefig(f"{plot_directory}PA_{FATable[fa].actuator_id}.png")
        #else:
        #    plt.savefig(f"{plot_directory}SA_{secondary_actuator_id[fa]}.png")

            
def main(t_start, t_end):

    loop_over_actuators(
        client, "primaryCylinderFollowingError", len(FATable), t_start, t_end
    )

    loop_over_actuators(
        client, "secondaryCylinderFollowingError", 112, t_start, t_end
    )
 